## Functionality 3 - Shortest Ordered Route 

In [1]:
#from google.colab import drive
#drive.mount('/content/drive/')

In [2]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gzip
import json
import time
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

In [3]:
class Relation:
    def __init__(self, type_relation, time, source, target, weight):
        self.type_relation_ = type_relation
        self.time_ = time
        self.source_ = source.get_ID
        self.target_ = target.get_ID
        self.weight_ = weight

    @property
    def get_type(self):
        return self.type_relation_
    
    @property
    def time(self):
        return self.time_
    
    @property
    def target(self):
        return self.target_
    
    @property
    def source(self):
        return self.source_
    
    @property
    def weight(self):
        return self.weight_
    
    def set_weight(self, weight):
        self.weight_ = weight
    
    def __str__(self): 
        return "{\"type_relation\": \"" + self.type_relation_ + "\", \"time\": " + str(self.time_) + ", \"source\": " + str(self.source_) + ", \"target\": " + \
        str(self.target_) + ", \"weight\": "+ str(self.weight_) + "}"
    
    def __repr__(self): 
        return self.__str__()
    
    
     

class User:
    def __init__(self, ID_user):
        self.ID_user = ID_user
        self.in_relation = dict()
        self.out_relation = dict()

    def add_in_relation(self, in_relation):
        if in_relation.time in self.in_relation:
            if in_relation.get_type not in self.in_relation[in_relation.time]:
                self.in_relation[in_relation.time][in_relation.get_type] = []
        else:
            self.in_relation[in_relation.time] = {in_relation.get_type: []}
        self.in_relation[in_relation.time][in_relation.get_type].append(in_relation)

    
    def add_out_relation(self, out_relation):
        if out_relation.time in self.out_relation:
            if out_relation.get_type not in self.out_relation[out_relation.time]:
                self.out_relation[out_relation.time][out_relation.get_type] = []
        else:
            self.out_relation[out_relation.time] = {out_relation.get_type: []}
        self.out_relation[out_relation.time][out_relation.get_type].append(out_relation)
    
    def set_in_relation(inRelations):
        self.in_relation = inRelations
    
    def set_out_relation(outRelation):
        self.out_relation = outRelation

    @classmethod
    def from_json(cls, json):
        return cls(json["ID_user"])
    
    @property
    def get_ID(self):
        return self.ID_user

    @property
    def get_in_relation(self):
        return self.in_relation
    
    @property
    def get_out_relation(self):
        return self.out_relation

    def __str__(self):
        return "{\"in_relation\": " + str(self.in_relation) +  ", \"out_relation\": " + str(self.out_relation) + "}"

    def to_json(self):
        return {"in_relation": self.in_relation, "out_relation":self.out_relation }

    def __repr__(self): 
        return self.__str__()

In [4]:
def getMinUnvisited(unvisited, dist):
    aux = {key: dist[key] for key in unvisited}
    minimum = min(aux.values())
    for key in unvisited:
        if dist[key] == minimum:
            return key

In [5]:
def convertDate(time):
    tmp = time.split("/")
    return int(tmp[1] + tmp[0])

In [6]:
def getShortestPath(source, target, prev, dist):
    path = [target]
    cost = dist[target]
    while target != source:
        path.append(prev[target])
        target = prev[target]
    path.reverse()
    return path, cost

In [7]:
def getNeighbors(node, graph, start, end):
    neighbors = []
    x = graph[node].get_out_relation
    for date in x.keys():
        if start <= date <= end:
            for rel in x[date].keys():
                neighbors.extend([x[date][rel][i].target for i in range(len(x[date][rel]))])
    return set(neighbors)

In [8]:
def overallWeight(source, target,start, end):
    w = 0
    x = graph[source].get_out_relation
    for date in x.keys():
        if start <= date <= end: 
            for rel in x[date].keys():
                for i in range(len(x[date][rel])):
                    if x[date][rel][i].target == target:
                        w += x[date][rel][i].weight
    return w

In [9]:
def myDijkstra(graph, source, target, start, end):
    unvisited = set(graph.keys())
    dist = dict()
    prev = dict()
    
    for u in unvisited:
        dist[u] = float('inf')
        prev[u] = -1
    
    dist[source] = 0    
    
    while len(unvisited) > 0: #TO ADD: loop not connected
        
        current_node = getMinUnvisited(unvisited, dist)
        unvisited.remove(current_node)
        neighbor = getNeighbors(current_node,graph, start, end)
        
        for u in unvisited.intersection(neighbor):
            new_dist = dist[current_node] + overallWeight(current_node,u, start, end)
            if new_dist < dist[u]:
                dist[u] = new_dist
                prev[u] = current_node 
                
    return getShortestPath(source, target, prev, dist)

In [10]:
def shortestOrderedRoute(graph, start, end, seq_users, p_1, p_n):
    nodes = [p_1] + seq_users + [p_n]
    path = [p_1]
    weight = 0
    start = convertDate(start)
    end = convertDate(end)
    for i in range(len(nodes)-1):
        seq, w = myDijkstra(graph, nodes[i], nodes[i+1], start, end)
        if w < float('inf'):
            path.extend(seq[1:])
            weight += w
        else:
            print("It is not possible to find the shortest ordered route because node", nodes[i], "and node", nodes[i+1], "are not connected!")
            return 
    return path, weight

In [11]:
p_1,p_n = 1, 49
orderedRoute = [17]
start = "08/2008"
end = "12/2008"

with open('graph (1).pickle', 'rb') as handle:
    graph = pickle.load(handle)
    
bestPath, cost = shortestOrderedRoute(graph,start, end,orderedRoute,p_1,p_n)

print("The shortest ordered route between", p_1, "and", p_n,"is:",bestPath, "\nand it costs:", cost)

The shortest ordered route between 1 and 49 is: [1, 49, 17, 49] 
and it costs: 3
